In [ ]:
import os

import scipy
import pandas as pd
import numpy as np


# This notebook pickle-izes an unzipped directory of Bigquery data.


DIRECTORY = "data/bigquery"
SMALLER_DIRECTORY = "data/bigquery_processed"
OUTPUT_PICKLE = "all_data.pkl"


In [ ]:


f = []
for (dirpath, dirnames, filenames) in os.walk(DIRECTORY):
    f.extend(filenames)
    break
filenames = [os.path.join(DIRECTORY, filename) for filename in f]
filenames = sorted(filenames)

other_irrelevant_columns = ["Response", "UncensoredResponse", "MiddleboxASName", 'PreviousMiddleboxIP', 'PreviousMiddleboxASN', 'PreviousMiddleboxCountry', 'MiddleboxIP', 'MiddleboxASN', 'MiddleboxCountry', 'NextMiddleboxIP', 'NextMiddleboxASN', 'NextMiddleboxCountry']

cutting down on dataset size
go thru each file and discard columns unneeded for feature extraction

dfs = []
count = 0
for filename in filenames:
    count += 1
    print(count)
    if count % 10 == 0:
        path = os.path.join(SMALLER_DIRECTORY, os.path.basename(filename))
        pd.concat(dfs).to_csv(path)
        dfs = []
    print(f"Reading {filename}...")
    df = pd.read_csv(filename, low_memory=False) # TODO: explicitly specify dtypes to make this run much faster
    application_columns = [col for col in list(df.columns) if col.startswith("Censys") or col.startswith("Zgrab") ]
    df = df.drop(application_columns + other_irrelevant_columns, 1)
    dfs.append(df)

# combine all the resultant processed files into one big pickle

f = []
for (dirpath, dirnames, filenames) in os.walk(SMALLER_DIRECTORY):
    f.extend(filenames)
    break
filenames = [os.path.join(SMALLER_DIRECTORY, filename) for filename in f]
filenames = sorted(filenames)
dfs = []
for filename in filenames:
    dfs.append(pd.read_csv(filename, low_memory=False))
dfs = pd.concat(dfs)
dfs.to_pickle(OUTPUT_PICKLE)